# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab_to_int = {}
    int_to_vocab = {}

    words = set(text)

    for index, word in enumerate(words):
        vocab_to_int[word] = index
        int_to_vocab[index] = word

    # return tuple
    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
        
    dict = {
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quotation_Mark||",
        ";": "||Semicolon||",
        "!":  "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Parentheses||",
        ")": "||Right_Parentheses||",
        "-": "||Dash||",
        "\n": "||Return||"
    }

    return dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # Get the number of batches we can make
    n_batches = len(words) // batch_size
    # Keep only enough characters to make full batches
    words = words[:n_batches * batch_size]
    
    x, y = [], []
    
    for idx_start in range(0, len(words) - sequence_length):
        idx_end = sequence_length + idx_start

        x_batch = words[idx_start:idx_end]
        y_batch = words[idx_end]

        x.append(x_batch)
        y.append(y_batch)

    data = TensorDataset(torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y)))
    data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)

    # return a data loader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 16,  17,  18,  19,  20],
        [ 17,  18,  19,  20,  21],
        [ 26,  27,  28,  29,  30],
        [ 31,  32,  33,  34,  35],
        [ 34,  35,  36,  37,  38],
        [ 29,  30,  31,  32,  33],
        [ 32,  33,  34,  35,  36],
        [  5,   6,   7,   8,   9],
        [ 41,  42,  43,  44,  45],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([ 21,  22,  31,  36,  39,  34,  37,  10,  46,  14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [10]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # define model layers
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.n_layers, dropout=self.dropout, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   

        # return one batch of output word scores and the hidden state
        batch_size = nn_input.size(0)

        # See comment below for an explanation of the call of the long function on nn_input
        embeddings = self.embedding(nn_input.long())
        lstm_output, hidden = self.lstm(embeddings, hidden)

        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)

        lstm_output = self.fc(lstm_output)

        lstm_output = lstm_output.view(batch_size, -1, self.output_size)

        output = lstm_output[:, -1]

        # return one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


In [3]:
import sys
print(sys.version)
import torch
print(torch.__version__)

3.6.3 | packaged by conda-forge | (default, Dec  9 2017, 04:28:46) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
0.4.0


Why is there a call of the long method on the nn_input argument ?

`embeddings = self.embedding(nn_input.long()`

I prefer to develop Python code on my notebook using PyCharme due to its improved debugging capabilities. On my local machine I use Python 3.7.2 and PyTorch 1.1.0. The Jupyter notebook in project workspace uses Python 3.6.3 and PyTorch 0.4.0 (A very early version of PyTorch). The call of the long() method is not necessary in case PyTorch 0.4 is used but without this call a runtime exception will be generated with a more recent PyTorch version. In order to write code that can be used with a more modern version I added the call to the long method.

### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])

    # perform back propagation and optimization
    # zero accumulated gradients
    rnn.zero_grad()

    # get the output from the RNN model
    output, hidden = rnn(inp, hidden)
    
    # perform back propagation and optimization
    # calculate the loss and perform back prop
    loss = criterion(output, target)
    loss.backward()
    
    nn.utils.clip_grad_norm_(rnn.parameters(), 3)
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [23]:
# Data params
# Sequence Length
sequence_length =  5 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)


In [26]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn_2', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 4.689792463660241

Epoch:    1/10    Loss: 4.2506486055850985

Epoch:    1/10    Loss: 4.119766900062561

Epoch:    2/10    Loss: 3.9193006922613898

Epoch:    2/10    Loss: 3.8512874813079834

Epoch:    2/10    Loss: 3.8357031614780426

Epoch:    3/10    Loss: 3.686481646209393

Epoch:    3/10    Loss: 3.658941691040993

Epoch:    3/10    Loss: 3.6722753283977507

Epoch:    4/10    Loss: 3.543836343159894

Epoch:    4/10    Loss: 3.5227619446516036

Epoch:    4/10    Loss: 3.5427541996240617

Epoch:    5/10    Loss: 3.4319334075136005

Epoch:    5/10    Loss: 3.4152429865598677

Epoch:    5/10    Loss: 3.457053666472435

Epoch:    6/10    Loss: 3.3478050129111563

Epoch:    6/10    Loss: 3.3374331164360047

Epoch:    6/10    Loss: 3.3756693811416625

Epoch:    7/10    Loss: 3.271677585062634

Epoch:    7/10    Loss: 3.268126743555069

Epoch:    7/10    Loss: 3.318865019083023

Epoch:    8/10    Loss: 3.2156727866663766

Epoch:    8/1

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [28]:
# Data params
# Sequence Length
sequence_length =  10 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)


In [29]:
# Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn_3', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 4.800155313372612

Epoch:    1/15    Loss: 4.295316607236862

Epoch:    1/15    Loss: 4.149648283720016

Epoch:    2/15    Loss: 3.960336420735259

Epoch:    2/15    Loss: 3.8648463886976243

Epoch:    2/15    Loss: 3.840922714948654

Epoch:    3/15    Loss: 3.7163237276263636

Epoch:    3/15    Loss: 3.649523294329643

Epoch:    3/15    Loss: 3.666649709224701

Epoch:    4/15    Loss: 3.54705061697253

Epoch:    4/15    Loss: 3.508591303706169

Epoch:    4/15    Loss: 3.5438685443401337

Epoch:    5/15    Loss: 3.42458190469729

Epoch:    5/15    Loss: 3.4013562223911284

Epoch:    5/15    Loss: 3.434163553237915

Epoch:    6/15    Loss: 3.3267457640235314

Epoch:    6/15    Loss: 3.3014660474061968

Epoch:    6/15    Loss: 3.3514804376363756

Epoch:    7/15    Loss: 3.249208868915501

Epoch:    7/15    Loss: 3.240672340631485

Epoch:    7/15    Loss: 3.2787560139894487

Epoch:    8/15    Loss: 3.186591775912159

Epoch:    8/15    Lo

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
# Data params
# Sequence Length
sequence_length =  15 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

# Training parameters
# Number of Epochs
num_epochs = 25
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from workspace_utils import active_session # To avoid workspace going to sleep with active process

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

with active_session():
    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/trained_rnn_4', trained_rnn)
    print('Model Trained and Saved')

Training for 25 epoch(s)...
Epoch:    1/25    Loss: 4.736059062719345

Epoch:    1/25    Loss: 4.251644201874733

Epoch:    1/25    Loss: 4.101633416056633

Epoch:    2/25    Loss: 3.9062186076313337

Epoch:    2/25    Loss: 3.8182632459402086

Epoch:    2/25    Loss: 3.8058226590156554

Epoch:    3/25    Loss: 3.654737638897009

Epoch:    3/25    Loss: 3.607908751010895

Epoch:    3/25    Loss: 3.6262243562936782

Epoch:    4/25    Loss: 3.481361703329652

Epoch:    4/25    Loss: 3.4572589123249053

Epoch:    4/25    Loss: 3.491483371734619

Epoch:    5/25    Loss: 3.3608966653719747

Epoch:    5/25    Loss: 3.329722940683365

Epoch:    5/25    Loss: 3.375304072380066

Epoch:    6/25    Loss: 3.262109057279931

Epoch:    6/25    Loss: 3.2328515434265137

Epoch:    6/25    Loss: 3.283464374899864

Epoch:    7/25    Loss: 3.1740288277359987

Epoch:    7/25    Loss: 3.158056159257889

Epoch:    7/25    Loss: 3.2138303856849673

Epoch:    8/25    Loss: 3.100304522363323

Epoch:    8/25   

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [16]:
# Data params
# Sequence Length
sequence_length =  10 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

# Training parameters
# Number of Epochs
num_epochs = 25
# Learning Rate
learning_rate = 0.002

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from workspace_utils import active_session # To avoid workspace going to sleep with active process

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

with active_session():
    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./save/trained_rnn_5', trained_rnn)
    print('Model Trained and Saved')

Training for 25 epoch(s)...
Epoch:    1/25    Loss: 4.74056222653389

Epoch:    1/25    Loss: 4.278705691933632

Epoch:    1/25    Loss: 4.159188337087631

Epoch:    2/25    Loss: 3.9846062804810765

Epoch:    2/25    Loss: 3.8945005029439925

Epoch:    2/25    Loss: 3.888404146194458

Epoch:    3/25    Loss: 3.754388099491114

Epoch:    3/25    Loss: 3.7152307931184767

Epoch:    3/25    Loss: 3.7297460899353028

Epoch:    4/25    Loss: 3.626190343314425

Epoch:    4/25    Loss: 3.603180058479309

Epoch:    4/25    Loss: 3.6279492505788804

Epoch:    5/25    Loss: 3.528228174885007

Epoch:    5/25    Loss: 3.5129153014421464

Epoch:    5/25    Loss: 3.548646330952644

Epoch:    6/25    Loss: 3.4702093649424635

Epoch:    6/25    Loss: 3.4467398109436034

Epoch:    6/25    Loss: 3.49612506043911

Epoch:    7/25    Loss: 3.418127901149246

Epoch:    7/25    Loss: 3.399686658501625

Epoch:    7/25    Loss: 3.440955346107483

Epoch:    8/25    Loss: 3.3671395356925027

Epoch:    8/25    L

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:**

First I tried to train the network with a batch size of 64 and a sequence length of 10. The initial learning rate of 0.01 was reduced to 0.001 due to observed instabilities during the training process. 

The starting value of 10 for the sequence length was motivated by an guess of the average number of words in english sentences.
Later I tried to increase this value, since the actual average number of words is 14.4 (And has declined during the last 500 years !!):
https://medium.com/@theacropolitan/sentence-length-has-declined-75-in-the-past-500-years-2e40f80f589f

The value of 300 for the embedding dimension was motivated by the high loss of the initial experiment with an embedding dimension of 100 and recommendations found here: https://datascience.stackexchange.com/questions/51404/word2vec-how-to-choose-the-embedding-size-parameter

The increase of the hidden dimension from 128 to 256 and finally to 256 was motivated by the poor performance of trainings with a small hidden dimension.

I used the following combination of hyperparameters:

| Batch Size | Sequence Length | Number of Epochs | Learning Rate | Embedding Dimension | Hidden Dimension | Number of Layers | Loss |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 64 | 10 | 10 | 0.01 | 100 | 128 | 2 | 4.2 |
| 64 | 10 | 10 | 0.001 | 300 | 256 | 2 | 3.7 |
| 64 | 10 | 10 | 0.001 | 300 | 512 | 3 | 4.02 |
| 64 | 10 | 10 | 0.001 | 300 | 512 | 2 | 3.68 |

I interrupted the training process before the full number of epochs was reached since the loss stopped declining quite early.

Signifcant better results could be achieved after increasing the batch size to 128 and reducing the sequence length to 5. Learning rate, embedding dimension, hidden dimension and number of layers were kept constant. In order to check whether the sequence length or the batch size was responsible for the improvement, only the batch size was increased to a value of 128, the sequence length was kept at 10.


| Batch Size | Sequence Length | Number of Epochs | Learning Rate | Embedding Dimension | Hidden Dimension | Number of Layers | Loss |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 128 | 5 | 10 | 0.001 | 300 | 512 | 2 | 3.17 |
| 128 | 10 | 15 | 0.001 | 300 | 512 | 2 | 2.96 |

The second line within the table above shows the positive effect of the batch size.

Due to the small number of epochs, the training stopped before the loss reached its minimum. 2 additional trainings were started with a larger number of epochs and in the first case an increased sequence length:

| Batch Size | Sequence Length | Number of Epochs | Learning Rate | Embedding Dimension | Hidden Dimension | Number of Layers | Loss |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 128 | 15 | 25 | 0.001 | 300 | 512 | 2 | 2.70 |
| 128 | 10 | 25 | 0.002 | 300 | 512 | 2 | 3.12 |

The last training was done with an slightly increased learning rate of 0.002.

The last 2 trained neural nets (trained_rnn_4 and trained_rnn_5) were used to generate TV scripts. A visual inspection of the generated texts revealed no significant differences, although the net trained with the shorter sequence length seemed to generate word repetitions ("hey, hey, hey", "No, No, No") more frequently.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [31]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'george' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

george: nudging georges daughter against the wall of the cycle?

kramer: no no no, that's no sweet'ning superior. i don't even want to talk about it.

george: i gotta go someone's with you.

elaine: i don't want to get it.(to elaine) i mean, i think it's possible.

george: you know, i was just wondering if i was a close seed.

kramer:(to jerry) i can't. i got a big problem with you.

george: well, i don't have any ideas finding it.

george: i don't even know where that is!

jerry: i don't know if i could talk to him, he doesn't have a job. you know what happens to me, they were just tendencies. you know, i gotta tell you, if i can't deliver my apartment keys, it's all very exciting.

jerry: oh! i don't want any scenes here!

kramer: hey! hey, hey! hey!

jerry: what is this? what?

elaine: you were snooping..

jerry:(interrupting) what?

george: you know, it's a very awkward situation.

jerry: well, i guess we should.

george: what do you mean?

kramer:(shouts) oh, you know, i gotta go,

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [32]:
# save script to a text file
f =  open("generated_script_george_4.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.